In [1]:
from __future__ import print_function
import os

os.environ["OPENBLAS_NUM_THREADS"] = "1"

import random
from optparse import OptionParser
import pandas as pd

from akita_utils.tsv_gen_utils import (
    filter_boundary_ctcfs_from_h5,
    filter_by_rmsk,
    filter_by_ctcf,
    add_orientation,
    add_background,
    add_diff_flanks_and_const_spacer,
    filter_sites_by_score,
    add_fixed_core_coordinates,
    validate_df_lenght_multiplicative,
)

2022-12-15 09:21:59.137839: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2022-12-15 09:21:59.137871: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# parameters
fixed_core_num_motifs = 2
flank_sets_num_motifs =10
orientation_string = ">>"
flank_range = "0,30"
flank_spacer_sum = 90
backgrounds_indices = "1"
jaspar_file = "/project/fudenber_735/motifs/mm10/jaspar/MA0139.1.tsv.gz"
rmsk_file = "/project/fudenber_735/genomes/mm10/database/rmsk.txt.gz"
all_permutations = False

In [3]:
orient_list = orientation_string.split(",")
num_orients = len(orient_list)
N = len(orient_list[0])

flank_start, flank_end = [
    int(num) for num in flank_range.split(",")
]

rmsk_exclude_window = flank_end
ctcf_exclude_window = 2 * flank_end

background_indices_list = backgrounds_indices.split(",")

if all_permutations == True:
    assert len(orient_list) == 1
    num_orients = 2**N

In [4]:
# loading motifs
score_key = "SCD"
weak_thresh_pct = 1
strong_thresh_pct = 99

In [5]:
sites = filter_boundary_ctcfs_from_h5(
    h5_dirs="/project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model*/scd.h5",
    score_key=score_key,
    threshold_all_ctcf=5,
)

sites = filter_by_rmsk(
    sites,
    rmsk_file=rmsk_file,
    exclude_window=rmsk_exclude_window,
    verbose=True,
)

sites = filter_by_ctcf(
    sites,
    ctcf_file=jaspar_file,
    exclude_window=ctcf_exclude_window,
    verbose=True,
)

annotating each site with boundary-wide scores
filtering sites by overlap with rmsk
filtering sites by overlap with ctcfs


In [6]:
core_sites = filter_sites_by_score(
    sites,
    score_key=score_key,
    upper_threshold=strong_thresh_pct,
    lower_threshold=weak_thresh_pct,
    mode="head",
    num_sites=fixed_core_num_motifs,
)

flanks_sites = filter_sites_by_score(
    sites,
    score_key=score_key,
    upper_threshold=strong_thresh_pct,
    lower_threshold=weak_thresh_pct,
    mode="head",
    num_sites=flank_sets_num_motifs,
)

In [7]:
core_sites_df = (
    core_sites[["chrom", "start_2", "end_2", "strand_2", score_key]]
    .copy()
    .rename(
        columns={
            "start_2": "start",
            "end_2": "end",
            "strand_2": "strand",
            score_key: "genomic_" + score_key,
        }
    )
)
core_sites_df.reset_index(drop=True, inplace=True)
core_sites_df.reset_index(inplace=True)

In [8]:
flanks_sites_df = (
    flanks_sites[["chrom", "start_2", "end_2", "strand_2", score_key]]
    .copy()
    .rename(
        columns={
            "start_2": "start",
            "end_2": "end",
            "strand_2": "strand",
            score_key: "genomic_" + score_key,
        }
    )
)
flanks_sites_df.reset_index(drop=True, inplace=True)
flanks_sites_df.reset_index(inplace=True)

In [9]:
# def concatenate_strings(list_of_strings):
#     output_string = ""
#     for input_string in list_of_strings:
#         output_string = output_string + "," + str(input_string)
#     return output_string[1:]

In [ ]:
# def add_fixed_core_coordinates(flanks_sites_df, core_sites_df):
    
#     rep_unit = flanks_sites_df.copy()
#     comma_separated_df = pd.concat([rep_unit for i in range(fixed_core_num_motifs)], ignore_index=True)

#     core_chrom, core_start, core_end, core_strand, core_genomic_SCD = [], [], [], [], []
#     for s in core_sites_df.itertuples():
#         for i in range(flank_sets_num_motifs):
#             core_chrom.append(s.chrom)
#             core_start.append(s.start)
#             core_end.append(s.end)
#             core_strand.append(s.strand)
#             core_genomic_SCD.append(s.genomic_SCD)
    
#     comma_separated_df = comma_separated_df.rename(columns={"chrom": "flank_chrom", 
#                                              "start": "flank_start",
#                                              "end": "flank_end",
#                                              "strand": "flank_strand",
#                                             "genomic_SCD": "flank_genomic_SCD"})
    
#     comma_separated_df["core_chrom"] = core_chrom
#     comma_separated_df["core_start"] = core_start
#     comma_separated_df["core_end"] = core_end
#     comma_separated_df["core_strand"] = core_strand
#     comma_separated_df["core_genomic_SCD"] = core_genomic_SCD
    
#     comma_separated_df["chrom"] = comma_separated_df.apply(lambda x: concatenate_strings([x.core_chrom, x.flank_chrom]), axis=1)
#     comma_separated_df["start"] = comma_separated_df.apply(lambda x: concatenate_strings([x.core_start, x.flank_start]), axis=1)
#     comma_separated_df["end"] = comma_separated_df.apply(lambda x: concatenate_strings([x.core_end, x.flank_end]), axis=1)
#     comma_separated_df["strand"] = comma_separated_df.apply(lambda x: concatenate_strings([x.core_strand, x.flank_strand]), axis=1)
#     comma_separated_df["genomic_SCD"] = comma_separated_df.apply(lambda x: concatenate_strings([x.core_genomic_SCD, x.flank_genomic_SCD]), axis=1)
    
#     comma_separated_df = comma_separated_df.drop(columns=["core_chrom", "core_start", "core_end", "core_strand", "core_genomic_SCD"])
#     comma_separated_df = comma_separated_df.drop(columns=["flank_chrom", "flank_start", "flank_end", "flank_strand", "flank_genomic_SCD"])
    
#     return comma_separated_df
    

In [10]:
comma_separated_df = add_fixed_core_coordinates(flanks_sites_df, core_sites_df)

In [11]:
comma_separated_df

,index,chrom,start,end,strand,genomic_SCD
0,0,"chr2,chr2","48886252,48886252","48886271,48886271","-,-","53.6875,53.6875"
1,1,"chr2,chr12","48886252,35192359","48886271,35192378","-,-","53.6875,53.65625"
2,2,"chr2,chr8","48886252,102781112","48886271,102781131","-,-","53.6875,53.5"
3,3,"chr2,chr5","48886252,49961991","48886271,49962010","-,-","53.6875,53.3125"
4,4,"chr2,chr9","48886252,26776356","48886271,26776375","-,-","53.6875,53.125"
5,5,"chr2,chr9","48886252,51152589","48886271,51152608","-,-","53.6875,53.0625"
6,6,"chr2,chr8","48886252,58566264","48886271,58566283","-,+","53.6875,52.875"
7,7,"chr2,chr12","48886252,40837731","48886271,40837750","-,+","53.6875,52.84375"
8,8,"chr2,chr6","48886252,101261721","48886271,101261740","-,-","53.6875,52.5"
9,9,"chr2,chr8","48886252,36523116","48886271,36523135","-,-","53.6875,52.5"


In [12]:
df_with_orientation = add_orientation(
    comma_separated_df,
    orientation_strings=orient_list,
    all_permutations=all_permutations,
)

In [13]:
df_with_background = add_background(
    df_with_orientation, background_indices_list
)

In [14]:
df_with_flanks_spacers = add_diff_flanks_and_const_spacer(
    df_with_background, flank_start, flank_end, flank_spacer_sum
)

In [15]:
df_with_flanks_spacers

,index,chrom,start,end,strand,genomic_SCD,orientation,background_index,flank_bp,spacer_bp
0,0,"chr2,chr2","48886252,48886252","48886271,48886271","-,-","53.6875,53.6875",>>,1,0,90
1,1,"chr2,chr12","48886252,35192359","48886271,35192378","-,-","53.6875,53.65625",>>,1,0,90
2,2,"chr2,chr8","48886252,102781112","48886271,102781131","-,-","53.6875,53.5",>>,1,0,90
3,3,"chr2,chr5","48886252,49961991","48886271,49962010","-,-","53.6875,53.3125",>>,1,0,90
4,4,"chr2,chr9","48886252,26776356","48886271,26776375","-,-","53.6875,53.125",>>,1,0,90
...,...,...,...,...,...,...,...,...,...,...
615,5,"chr12,chr9","35192359,51152589","35192378,51152608","-,-","53.65625,53.0625",>>,1,30,60
616,6,"chr12,chr8","35192359,58566264","35192378,58566283","-,+","53.65625,52.875",>>,1,30,60
617,7,"chr12,chr12","35192359,40837731","35192378,40837750","-,+","53.65625,52.84375",>>,1,30,60
618,8,"chr12,chr6","35192359,101261721","35192378,101261740","-,-","53.65625,52.5",>>,1,30,60


In [17]:
df_with_flanks_spacers = df_with_flanks_spacers.drop(columns="index")

In [18]:
df_with_flanks_spacers.index.name = "experiment_id"

In [19]:
df_with_flanks_spacers

,chrom,start,end,strand,genomic_SCD,orientation,background_index,flank_bp,spacer_bp
experiment_id,,,,,,,,,
0,"chr2,chr2","48886252,48886252","48886271,48886271","-,-","53.6875,53.6875",>>,1,0,90
1,"chr2,chr12","48886252,35192359","48886271,35192378","-,-","53.6875,53.65625",>>,1,0,90
2,"chr2,chr8","48886252,102781112","48886271,102781131","-,-","53.6875,53.5",>>,1,0,90
3,"chr2,chr5","48886252,49961991","48886271,49962010","-,-","53.6875,53.3125",>>,1,0,90
4,"chr2,chr9","48886252,26776356","48886271,26776375","-,-","53.6875,53.125",>>,1,0,90
...,...,...,...,...,...,...,...,...,...
615,"chr12,chr9","35192359,51152589","35192378,51152608","-,-","53.65625,53.0625",>>,1,30,60
616,"chr12,chr8","35192359,58566264","35192378,58566283","-,+","53.65625,52.875",>>,1,30,60
617,"chr12,chr12","35192359,40837731","35192378,40837750","-,+","53.65625,52.84375",>>,1,30,60


In [20]:
(expected_df_len, observed_df_len) = validate_df_lenght_multiplicative(
        fixed_core_num_motifs,
        flank_sets_num_motifs,
        num_orients,
        len(background_indices_list),
        (flank_end - flank_start + 1),
        df_with_flanks_spacers,
    )

In [21]:
expected_df_len

620